In [3]:
import pandas as pd
import numpy as np
import itertools
cars_data = pd.read_csv("vehicles_cleaned_train.csv")
cars_data_teste = pd.read_csv("vehicles_cleaned_train.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'vehicles_cleaned_train.csv'

In [ ]:
def RL(data, data_teste, combination, combination_teste):

    #PARAMETROS
    d = data['price'].values.reshape(-1, 1)
    # N representa o número de dados para o cálculo da RL
    N = d.shape[0]
    # X é a matriz dos dados
    X = np.hstack([np.ones((N, 1)), combination])
    # R é o produto das matrizes X.T e X
    R = X.T @ X
    # o vetor p é dado pelo produto entre a matriz X.T e o vetor d
    p = X.T @ d
    # Calcule a solução wo e o erro e
    wo = np.linalg.solve(R, p)
    e = d - X @ wo

    #TESTE
    d_teste = data_teste['price'].values.reshape(-1, 1)
    # N representa o número de dados para o cálculo da RL
    N_teste = d_teste.shape[0]
    # X é a matriz dos dados
    X_teste = np.hstack([np.ones((N_teste, 1)), combination_teste])
    # previsoes e erros
    previsto = []
    erro = []
    # para cada linha do conjunto de teste
    for i in range(len(data_teste)):
        # extrai as features da linha i
        xTeste_i = X_teste[i, :]
        # calcula o valor predito pelo modelo
        y_pred_i = xTeste_i @ wo
        # calcula o erro em relação ao valor real
        e_i = d[i] - y_pred_i
        # salva nos vetores
        previsto.append(y_pred_i)
        erro.append(e_i)
        # imprime o valor predito, o valor real e o erro
        # print(f"Valor predito: {y_pred_i}, Valor real: {dTeste[i]}, Erro: {e_i}")
    # erro da combinacao
    mse = np.mean((d_teste - previsto)**2)*(1/N_teste)
    return mse

In [ ]:
# TREINO
# Extrai as colunas relevantes dos dados de entrada
fabrication = cars_data['year'].values.reshape(-1, 1)
odometer = cars_data['odometer'].values.reshape(-1, 1)
# Cria as variáveis dummy para as variáveis categóricas
condition_dummy = pd.get_dummies(cars_data['condition'])
cylinders_dummy = pd.get_dummies(cars_data['cylinders'])
fuel_dummy = pd.get_dummies(cars_data['fuel'])
transmission_dummy = pd.get_dummies(cars_data['transmission'])
size_dummy = pd.get_dummies(cars_data['size'])
type_dummy = pd.get_dummies(cars_data['type'])
# Nossas novas variáveis
# 1. Idade do carro ao inves de ano de fabricação
current_year = 2023
idade = current_year - fabrication
# 2. Milhagem por anos ao invés de só milhagem
milhagem_anos = odometer/idade
x = np.hstack([fabrication, odometer, condition_dummy, cylinders_dummy, fuel_dummy, transmission_dummy, size_dummy, type_dummy])

#TESTE
# Extrai as colunas relevantes dos dados de entrada
fabrication_teste = cars_data_teste['year'].values.reshape(-1, 1)
odometer_teste = cars_data_teste['odometer'].values.reshape(-1, 1)
# Cria as variáveis dummy para as variáveis categóricas
condition_dummy_teste = pd.get_dummies(cars_data_teste['condition'])
cylinders_dummy_teste = pd.get_dummies(cars_data_teste['cylinders'])
fuel_dummy_teste = pd.get_dummies(cars_data_teste['fuel'])
transmission_dummy_teste = pd.get_dummies(cars_data_teste['transmission'])
size_dummy_teste = pd.get_dummies(cars_data_teste['size'])
type_dummy_teste = pd.get_dummies(cars_data_teste['type'])
# Nossas novas variáveis
# 1. Idade do carro ao inves de ano de fabricação
current_year = 2023
idade_teste = current_year - fabrication_teste
# 2. Milhagem por anos ao invés de só milhagem
milhagem_anos_teste = odometer_teste/idade_teste

mse_dict = {}

for i in range(1, 3):
    for combination in itertools.combinations(x, i):
        mse = RL(cars_data, cars_data_teste, np.hstack([fabrication, idade, odometer, milhagem_anos, condition_dummy, cylinders_dummy, fuel_dummy, transmission_dummy, size_dummy, type_dummy]), np.hstack([fabrication_teste, idade_teste, odometer_teste, milhagem_anos_teste, condition_dummy_teste, cylinders_dummy_teste, fuel_dummy_teste, transmission_dummy_teste, size_dummy_teste, type_dummy_teste]))
        mse_dict[combination] = mse

